In [ ]:
import numpy as np
import pandas as pd
#import fiona 
import math
#import xarray as xa
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from datetime import datetime
#import random
#from scipy.interpolate import interp1d
import geopandas as gpd

In [ ]:
#radadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_GF_v1.1.csv'
#radadj_solf_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_solf_GF_v1.1.csv'
#pptadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_pptadj_GF_v1.1.csv'
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2f.gdb'

markstro_param_path = "c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_markstro/"
path = 'c:/Users/markstro/work1.1/weather/'
farns_v11_path = 'c:/Users/markstro/work1.1/GIS/farns_gf_1_1_mm_per_month.shp'
jh_coef_hru_fn = 'c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_markstro/jh_coef_hru.csv'
nrel_shapefile_fn = "c:/Users/markstro/work1.1/GIS/nhm_shapefiles/nhm_hru_gf1_1_nrel_solrad.shp"
tmax_path = 'c:/Users/markstro/work1.1/cbh_gridmet/tmax.cbh'
tmin_path = 'c:/Users/markstro/work1.1/cbh_gridmet/tmin.cbh'

In [ ]:
# Read the v11 Farnsworth shapefile
# The PET values (cols jan - dec) are in units of mm / month
farns_v11 = gpd.read_file(farns_v11_path)

In [ ]:
farns_v11.head()

In [ ]:
days_in_month = [31,28,31,30,31,30,31,31,30,31,30,31]

In [ ]:
farns_vals = farns_v11.iloc[:,4:16].values
farns_vals[farns_vals < 1.0] = 1.0
print(min(farns_vals.flatten()), max(farns_vals.flatten()))
print(farns_vals)

In [ ]:
# Extract the PET values and convert from mm/month to inches/day

#farns_vals = farns_v11[['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']].values
#farns_vals = farns_v11.iloc[:,4:16].values

#farns_vals = farns_v11.iloc[:,8:20].values

nhru = farns_vals.shape[0]
nmonth = farns_vals.shape[1]

for imon in range(nmonth):
    farns_vals[:,imon] = farns_vals[:,imon] / 25.4 / days_in_month[imon]
#    farns_vals[:,imon] = farns_vals[:,imon] / 25.4
    
print(nhru, nmonth)

In [ ]:
# units are inches per day
print(farns_vals)

In [ ]:
print(min(farns_vals.flatten()), max(farns_vals.flatten()))

In [ ]:
# Extract the Farnsworth v11 HRU order
farns_nhru_v11 = farns_v11['nhru_v11'].values
print(farns_nhru_v11)

In [ ]:
# read the jh_coef_hru parameters

# read the jh_coef_hru values from the csv file produced by jh_coef_hru.ipynb
jh_coef_hru = pd.read_csv(jh_coef_hru_fn)
jh_coef_hru.head()

In [ ]:
jh_coef_hru_vals = jh_coef_hru['jh_coef_hru'].values
jh_coef_hru_idx = jh_coef_hru['$id'].values
print(jh_coef_hru_vals)
print(jh_coef_hru_idx)

In [ ]:
# Read the cbh file
def read_cbh(cbh_fn):
#    foo = np.loadtxt(cbh_fn, delimiter=' ', skiprows=3,)
    foo = np.loadtxt(cbh_fn, skiprows=3,)

    dates = foo[:,0:6].astype('int')
    vals = foo[:,6:foo.shape[1]]
    
    return dates, vals

In [ ]:
dates, tmax_c = read_cbh(tmax_path)
#tmax_f = tmax_c * 1.8 +32

In [ ]:
#mon_cnt = 1
#last_mon = dates[0,1]

#for iday in range(1,dates.shape[0]):
#    if dates[iday,1] != last_mon:
#        mon_cnt += 1
#        last_mon = dates[iday,1]
        
#print(mon_cnt)


In [ ]:
print(tmax_c.shape)

nhru = tmax_c.shape[1]
ndays = tmax_c.shape[0]

mon_cnt = 12

tmax_c_monthly = np.zeros(mon_cnt * nhru)
tmax_c_monthly.shape = (mon_cnt, nhru)
monthly_count = np.zeros(mon_cnt * nhru, dtype=np.int)
monthly_count.shape = (mon_cnt, nhru)
print(tmax_c_monthly.shape)

#strt_yr = dates[0,0]
for iday in range(ndays):
    date = datetime(dates[iday,0],dates[iday,1],dates[iday,2])

#    jday = date.day - 1
#    iyr = date.year - strt_yr
#    imon = date.month - 1 + (iyr * 12)
    imon = date.month - 1
    for ihru in range(nhru):
        tmax_c_monthly[imon,ihru] += tmax_c[iday,ihru]
        monthly_count[imon, ihru] +=1


In [ ]:
print(tmax_c_monthly)
print(monthly_count)

In [ ]:
tmax_c_monthly = tmax_c_monthly / monthly_count

In [ ]:
print(tmax_c_monthly)

In [ ]:
del(tmax_c)

In [ ]:
dates, tmin_c = read_cbh(tmin_path)

In [ ]:
tmin_c_monthly = np.zeros(mon_cnt * nhru)
tmin_c_monthly.shape = (mon_cnt, nhru)
monthly_count = np.zeros(mon_cnt * nhru, dtype=np.int)
monthly_count.shape = (mon_cnt, nhru)
print(tmin_c_monthly.shape)

strt_yr = dates[0,0]
for iday in range(ndays):
    date = datetime(dates[iday,0],dates[iday,1],dates[iday,2])

#    jday = date.day - 1
#    iyr = date.year - strt_yr
#    imon = date.month - 1 + (iyr * 12)
    imon = date.month - 1
    for ihru in range(nhru):
        tmin_c_monthly[imon,ihru] += tmin_c[iday,ihru]
        monthly_count[imon, ihru] +=1


In [ ]:
tmin_c_monthly = tmin_c_monthly / monthly_count

In [ ]:
print(tmin_c_monthly)

In [ ]:
del(tmin_c)

In [ ]:
tmax_f_monthly = (tmax_c_monthly * 1.8) + 32
tmin_f_monthly = (tmin_c_monthly * 1.8) + 32
tavg_f_monthly = (tmax_f_monthly + tmin_f_monthly) / 2.0

In [ ]:
tavg_c_monthly = (tmax_c_monthly + tmin_c_monthly) / 2.0

In [ ]:
print(tavg_f_monthly)

In [ ]:
print(tavg_f_monthly.shape)

In [ ]:

# Read in the monthly short wave data from NREL. It's in the shapefile that was created by running zonal means
# on the monthly DNI geotifs downloaded from NREL.

nrel_solrad = gpd.read_file(nrel_shapefile_fn)

In [ ]:
nhru_v11_id = nrel_solrad["nhru_v11"].values
print(nhru_v11_id)

In [ ]:
nrel_solrad.head()

In [ ]:
# Look for nans in the NREL values and subsitute with interpolated value

print(nrel_solrad.shape)

count = 0
for ii in range(nrel_solrad.shape[0]):
    for jj in range(8,20):
        if np.isnan(nrel_solrad.iloc[ii,jj]):
#            print(" (", ii,jj, ")", end = '')
            count += 1
#print("")
print(count)
      

#for ii in range(nrel_solrad.shape[0]):
for jj in range(8,20):
    nrel_solrad.iloc[:,jj] = nrel_solrad.iloc[:,jj].interpolate()

count = 0
for ii in range(nrel_solrad.shape[0]):
    for jj in range(8,20):
        if np.isnan(nrel_solrad.iloc[ii,jj]):
#            print(" (", ii,jj, ")", end = '')
            count += 1

#print("")
print(count)

In [ ]:
nrel_solrad.head()

In [ ]:
# Extract the NREL monthly solrad targets from the dataframe into a numpy 2D array for the radadj calculation below.

#solrad_targets_vals= nrel_solrad[["dni_jan_me", "dni_feb_me", "dni_mar_me", "dni_apr_me", "dni_may_me", "dni_jun_me",
#                    "dni_jul_me", "dni_aug_me", "dni_sep_me", "dni_oct_me", "dni_nov_me", "dni_dec_me"]].values

solrad_targets_vals = nrel_solrad.iloc[:,8:20].values
print(solrad_targets_vals)

In [ ]:
for ii in range(solrad_targets_vals.shape[0]):
    for jj in range(solrad_targets_vals.shape[1]):
        if np.isnan(solrad_targets_vals[ii,jj]):
            print(ii, jj, solrad_targets_vals[ii,jj])

In [ ]:
# The NREL solrad values are in units of kWh/m2/Day. PRMS uses Langleys per day. The conversion factor comes from
# https://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/GEM/SolarRadConversion.pdf

#   1 kWh     m2      1 Langly  1000    1000 Lang          Lang
# --------- -------  -------    ----  = ---------  = 85.98 ----
#  m2-day   11.63 W   1 hour     1k     11.63 day          day

# units of the NREL data are w/m2
# so 1 Langley/day = 0.484583 Watt/m2
# so 1 watt/m2 = 2.06363 Lang/day
solrad_targets_vals_langleys = solrad_targets_vals * 2.06363
print(solrad_targets_vals_langleys)
print(solrad_targets_vals_langleys.shape)

In [ ]:
print(tavg_f_monthly.shape)

In [ ]:
# DO j = 1, Active_hrus
#          elh = (597.3-(0.5653*Tavgc(i)))*2.54
#          Potet(i) = Jh_coef(i, Nowmonth)*(Tavgf(i)-Jh_coef_hru(i))*Swrad(i)/elh
#          IF ( Potet(i)<0.0) Potet(i) = 0.0
#
# so
#          elh = (597.3-(0.5653*Tavgc(iday,ihru)))*2.54
#          Jh_coef(ihru, imon) = farns(ihru, imon) / ((Tavgf(iday,ihru)-Jh_coef_hru(ihru)) / Swrad(imon,ihru) * elh)

#ndays = tavg_f.shape[0]
#nhru = tavg_f.shape[1]

#jh_coef_by_day = np.zeros(nhru * ndays)
#jh_coef_by_day.shape = (nhru, ndays)

nmo = tavg_f_monthly.shape[0]

jh_coef = np.zeros(nhru * nmo)
jh_coef.shape = (nhru, nmo)

month_count = np.zeros(nmo)

for imon in range(nmo):
#    date = datetime(dates[iday,0],dates[iday,1],dates[iday,2])
#    imon = date.month - 1
#    jday = date.day - 1
#    print(date, imon, jday, iday)
    for ihru in range(nhru):
        v11_id = nhru_v11_id[ihru] - 1
        if tavg_c_monthly[imon, v11_id] == 0.0:
            tavg_c_monthly[imon, v11_id] = 1.0
            
        elh = (597.3 - (0.5653 * tavg_c_monthly[imon, v11_id])) * 2.54
        if elh == 0.0:
            print ("crap", imon, ihru, tavg_c_monthly[imon, v11_id], elh)
        # farns_values and solrad_targets_vals_langleys are already in v11_id mapping
        # tavg_f, jh_coef_hru_vals, jh_coef_by_day are in 1 to nhru mapping
#        div = (tavg_f[iday,v11_id] - jh_coef_hru_vals[v11_id])
        
        jh_coef[v11_id, imon] = farns_vals[ihru, imon] / (tavg_f_monthly[imon,v11_id] - jh_coef_hru_vals[v11_id]) / solrad_targets_vals_langleys[ihru,imon] * elh
        if jh_coef[v11_id, imon] == 0.0:
            print("2", v11_id, imon, farns_vals[ihru, imon], elh)
        #        month_count[imon] += 1
        #        print(imon, ihru, v11_id, jh_coef[v11_id, imon], farns_vals[ihru, imon], (tavg_f_monthly[imon,v11_id] - jh_coef_hru_vals[v11_id]), solrad_targets_vals_langleys[ihru,imon], elh)

# the values should come out in 1 to nhru order, not v11_id order.

#for imon in range(nmo):
#    date = datetime(dates[iday,0],dates[iday,1],dates[iday,2])
#    imon = date.month - 1
#    jday = date.day - 1
#    for ihru in range(nhru):
#        jh_coef[ihru, imon] += jh_coef[ihru, imon]
#        month_count[imon] += 1

#for imon in range(nmonth):
#jh_coef = jh_coef / month_count

#print(jh_coef)
        


In [ ]:
print(farns_vals[0,0])

In [ ]:
tavg_f_monthly[0,nhru_v11_id[0] - 1]

In [ ]:
jh_coef_hru_vals[nhru_v11_id[0] - 1]

In [ ]:
solrad_targets_vals_langleys[0,0]

In [ ]:
 elh = (597.3 - (0.5653 * tavg_c_monthly[0, nhru_v11_id[0] - 1])) * 2.54
print(elh)

In [ ]:
jh_c = farns_vals[0,0] / (tavg_f_monthly[0,nhru_v11_id[0] - 1] - jh_coef_hru_vals[nhru_v11_id[0] - 1]) / solrad_targets_vals_langleys[0,0] * elh
print(jh_c)

In [ ]:
print(jh_coef)
print(jh_coef.shape)

In [ ]:
print(min(jh_coef.flatten()), max(jh_coef.flatten()))

In [ ]:
f = open(markstro_param_path + 'jh_coef.csv', "w")

In [ ]:
f.write("$id,jh_coef\n")

In [ ]:
# The indexes being written are in the order of the values in the tmin ncf files. Once those are available, these will
# need to be reordered.
kk = 1
for ii in range(nmonth):
    for jj in range(nhru):
        f.write(str(kk) + "," + str(jh_coef[jj,ii]) + '\n')
        kk = kk + 1

In [ ]:
f.close()